# Librerías
Importación de todas las librerías necesarias para manipulación de datos, escalado, modelado, validación y evaluación. Se hace uso de pandas, numpy y módulos de scikit-learn.

In [3]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score


### Preparación y Limpieza de los Datos Socioeconómicos

Esta etapa se enfoca en preparar los datos que alimentarán el modelo de aprendizaje automático, garantizando su calidad y formato adecuados. A continuación, se describen los pasos clave:

---

#### 2. Cargar Datos

Se importan los archivos `train.csv` y `test.csv`, los cuales contienen información sobre los estudiantes.

* El conjunto `train` incluye la variable objetivo `RENDIMIENTO_GLOBAL`.
* El conjunto `test` será utilizado posteriormente para realizar predicciones.

---

#### 3. Selección de Columnas Socioeconómicas Relevantes

Se seleccionan cuatro variables clave relacionadas con el entorno familiar del estudiante:

* `FAMI_ESTRATOVIVIENDA`: Estrato socioeconómico del hogar.
* `FAMI_TIENEINTERNET`: Acceso a internet en el hogar.
* `FAMI_EDUCACIONPADRE` y `FAMI_EDUCACIONMADRE`: Nivel educativo de los padres.

Estas columnas, junto con el `ID` y la variable objetivo (`RENDIMIENTO_GLOBAL`), se conservan en los conjuntos de datos.

---

#### 4. Limpieza de Valores Atípicos o Raros

En los datos pueden aparecer valores codificados como `98`, `99`, o `999` que representan respuestas inválidas, desconocidas o no aplicables.
Estos valores se reemplazan por `NaN` (valores faltantes) para tratarlos adecuadamente en el siguiente paso.

---

#### 5. Imputación de Valores Faltantes

Se realiza una **imputación simple por moda**:

* Para cada columna, se calcula el valor más frecuente (moda).
* Ese valor se utiliza para rellenar los campos faltantes, tanto en el conjunto de entrenamiento como en el de prueba.

Este enfoque es útil cuando se trabaja con variables categóricas y se desea mantener la consistencia.

---

#### 6. Codificación One-Hot

Las variables categóricas no pueden ser procesadas directamente por la mayoría de los modelos.
Por eso, se aplica **One-Hot Encoding**, que convierte cada categoría en una nueva columna binaria (0 o 1).

Esta técnica evita supuestos de orden o magnitud entre categorías y prepara los datos para el modelo SVM, que es sensible a la representación de las variables.


In [4]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

cols = [
    'FAMI_ESTRATOVIVIENDA',
    'FAMI_TIENEINTERNET',
    'FAMI_EDUCACIONPADRE',
    'FAMI_EDUCACIONMADRE'
]
train = train[['ID'] + cols + ['RENDIMIENTO_GLOBAL']]
test = test[['ID'] + cols]

valores_raros = ['98', '99', '999', 98, 99, 999]
for col in cols:
    train[col] = train[col].replace(valores_raros, np.nan)
    test[col] = test[col].replace(valores_raros, np.nan)

for col in cols:
    modo = train[col].mode()[0]
    train[col] = train[col].fillna(modo)
    test[col] = test[col].fillna(modo)

def one_hot(df, col):
    return pd.concat([df.drop(columns=[col]), pd.get_dummies(df[col], prefix=col)], axis=1)

for col in cols:
    train = one_hot(train, col)
    test = one_hot(test, col)

test = test.reindex(columns=train.drop(columns=['RENDIMIENTO_GLOBAL']).columns, fill_value=0)

### Modelado, Evaluación y Predicción

En esta etapa se realiza la transformación de la variable objetivo, el entrenamiento del modelo SVM (Support Vector Machine), su validación, y finalmente se generan las predicciones para el conjunto de prueba.

---

#### Codificación de la Variable Objetivo

La variable `RENDIMIENTO_GLOBAL` es de tipo categórico con niveles: `bajo`, `medio-bajo`, `medio-alto` y `alto`.

* Se convierte a valores numéricos con un mapeo ordinal:

  * `bajo → 0`
  * `medio-bajo → 1`
  * `medio-alto → 2`
  * `alto → 3`

Esto facilita el uso del modelo SVM, que requiere variables numéricas.

---

#### Escalado de los Datos

Se normalizan las variables predictoras (`X`) usando **`StandardScaler`**.
Esto es esencial en SVM porque la escala de las variables puede afectar significativamente el rendimiento del modelo.

* `fit_transform` se aplica al conjunto de entrenamiento.
* `transform` se aplica al conjunto de prueba para mantener la coherencia.

---

#### División para Validación

Para evaluar el modelo antes de hacer predicciones finales, se divide el conjunto de entrenamiento en dos partes:

* 70% para entrenamiento (`Xtrain`, `ytrain`)
* 30% para validación (`Xval`, `yval`)

Esto permite probar el modelo en datos no vistos antes de hacer predicciones definitivas.

---

#### Entrenamiento del Modelo

Se entrena un modelo **SVM con kernel RBF** (`radial basis function`), que permite capturar relaciones no lineales en los datos.

Parámetros usados:

* `C=1`: penaliza errores de clasificación (trade-off bias/varianza).
* `gamma='scale'`: ajuste automático del parámetro del kernel.

---

#### Evaluación del Modelo

Se evalúa el rendimiento del modelo con:

* **Accuracy en el conjunto de validación** (`Xval` vs. `yval`).
* **Validación cruzada con 5 particiones (CV=5)** para estimar la generalización del modelo.
  Se imprime la media y desviación estándar del accuracy.

---

#### Predicción Final

Una vez validado el modelo, se reentrena con todo el conjunto de entrenamiento (`X_scaled`, `y`) para aprovechar toda la información disponible.

Se generan predicciones sobre el conjunto de prueba (`test_scaled`).

---

#### Generación de Archivo de Envío (`submission.csv`)

Se crea un archivo con las predicciones finales.

* Se revierte la codificación numérica a categorías originales (`0 → bajo`, etc.).
* Se guarda el archivo `submission.csv` con dos columnas: `ID` y `RENDIMIENTO_GLOBAL`.


In [ ]:
mapa = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
inv_mapa = {v: k for k, v in mapa.items()}
y = train['RENDIMIENTO_GLOBAL'].map(mapa)
X = train.drop(columns=['RENDIMIENTO_GLOBAL'])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
test_scaled = scaler.transform(test)

Xtrain, Xval, ytrain, yval = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

clf = SVC(kernel='rbf', C=1, gamma='scale', random_state=42)
clf.fit(Xtrain, ytrain)

yval_pred = clf.predict(Xval)
acc = accuracy_score(yval, yval_pred)
print(f"Accuracy en validación: {acc:.4f}")
scores = cross_val_score(clf, X_scaled, y, cv=5)
print(f"CV Accuracy: {scores.mean():.4f} ± {scores.std():.4f}")

clf.fit(X_scaled, y)
y_test_pred = clf.predict(test_scaled)

submission = pd.DataFrame({
    'ID': test['ID'],
    'RENDIMIENTO_GLOBAL': pd.Series(y_test_pred).map(inv_mapa)
})
submission.to_csv("submission.csv", index=False)
print(submission.head())
